# Notebook #4: Cálculo de Rentabilidad

- Con los datos extraídos y el modelo de *machine learning* que se ha entrenado para predecir el precio del alquiler, somos capaces de realizar el cálculo de rentabilidad con ayuda de distintas funciones.

- Importamos las librerías y soportes.

In [2]:
%load_ext autoreload
%autoreload 2

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los dfFrames

from tqdm import tqdm

# Ignorar los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

# Librería para el acceso a variables y funciones
# -----------------------------------------------------------------------
import sys
sys.path.append("../")
from src import soporte_rentabilidad as sr
from src import soporte_yolo as sy
from src import soporte_scoring as ss
from src import soporte_blip as sb
from src import soporte_mongo as sm

- Como primer paso, definimos una variable con las rutas de los transformers: el encoder, el scaler y el modelo Random Forest.

- También vamos a conectarnos a la base de datos de Mongo para obtener el los datos correspondientes a viviendas en venta.

- Utilizaremos la función `predecir_alquiler` para aplicar el modelo a la totalidad de viviendas en venta, que devuelve el dataframe con una nueva columna 'alquiler_predicho'. Sus únicos argumentos son el archivo pickle con las viviendas en venta y la variable con las rutas de los transformers.

In [3]:
paths_transformers = ["../transformers/target_encoder.pkl", "../transformers/scaler.pkl", "../transformers/model.pkl"]
bd = sm.conectar_a_mongo("ProyectoRentabilidad")
gdf = sm.importar_a_geodataframe(bd, "venta")
gdf = sr.predecir_alquiler(gdf, paths_transformers)

- Comprobamos el dataframe.

- Para cada vivienda, tenemos una columna que corresponde a una lista con las urls de las imágenes. Para poder identificar cuál corresponde a cada estancia, utilizaremos el modelo YOLOv11, en su versión x y de clasificación, por cuanto no nos interesa determinar la ubicación ni la forma de los objetos en las imágenes, solamente su existencia.

- Con la función `identificar_urls_habitaciones`, se añaden etiquetas a la cocina y al baño de cada anuncio. Para optimizar el tiempo de procesamiento, cuando las ha encontrado, no continua con el resto de imágenes, sino que salta al siguiente anuncio.

- El resultado es un dataframe con nuevas columnas para la URL del baño y de la cocina. Cuando uno de los dos valores es nulo, elimina esa fila. El return también incluye un dataframe con todas las detecciones de las imágenes procesadas, como una forma de poder validar los resultados.

In [4]:
gdf, detecciones = sy.identificar_urls_habitaciones(gdf, 'urls_imagenes', drop_nulls=True)

100%|██████████| 352/352 [53:41<00:00,  9.15s/it]  

Se han eliminado 116 filas donde 'url_cocina' o 'url_banio' son None o NaN.


- Vamos a comprobar el resultado del filtrado.

- Además del dataframe con las imágenes filtradas, tenemos un segundo dataframe como retorno, que almacena los elementos que se han detectado en cada imagen, la URL y si ha sido etiquetado como cocina o baño. Esto nos permite realizar generar una métrica sobre la efectividad de YOLO.

- De las 365 viviendas en venta, se han encontrado imágenes de baño y cocina para 266. Dado que el cliente objetivo del proyecto no es quién se dedica a reformas, sin poder conocer el estado de estas estancias, el cálculo de rentabilidad puede cambiar significativamente por ser los espacios que cuesta más dinero reformar.

- Por ese motivo, las eliminaremos del conjunto de datos con el parámetro *drop nulls*.

In [5]:
gdf.shape[0]

266

In [14]:
#gdf.to_pickle("../data/transformed/final_tags.pkl")

In [6]:
detecciones[detecciones['habitación']=='kitchen'].head(5)

,url,detecciones,habitación
2,https://img4.idealista.com/blur/WEB_LISTING-M/...,"[microwave, refrigerator, wardrobe, dishwasher...",kitchen
7,https://img4.idealista.com/blur/WEB_LISTING-M/...,"[microwave, washbasin, refrigerator, washer, s...",kitchen
13,https://img4.idealista.com/blur/WEB_LISTING-M/...,"[microwave, washer, stove, dishwasher, washbasin]",kitchen
14,https://img4.idealista.com/blur/WEB_LISTING-M/...,"[washer, microwave, washbasin, refrigerator, s...",kitchen
16,https://img4.idealista.com/blur/WEB_LISTING-M/...,"[washer, microwave, space_heater, dishwasher, ...",kitchen


- Al comprobar las URLs de forma manual utilizando una muestra aleatoria, se confirma una accuracy de entorno al 90% en el etiquetado.

In [7]:
for value in detecciones[detecciones['habitación']=='kitchen']['url'].sample(10):
    print(value)

https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/a5/95/64/918499997.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/2a/fb/58/1291969391.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/75/88/d3/1296526056.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/a1/d1/c1/1297365981.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/5d/6d/c6/1300123245.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/eb/af/3c/1290783297.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/ba/4f/f0/1300146068.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/8d/e4/1b/1298117937.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/2d/b8/9d/1300229160.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/53/8d/25/1296878330.webp


- Como una capa de verificación adicional, vamos a utilizar el modelo BLIP de Salesforce, que crea descripciones de las imágenes, en este caso, de las URLs identificadas por YOLO. Contamos luego el número de ocurrencias de 'kitchen' o 'bathroom'.

In [8]:
captions_cocinas = sb.generar_descripciones(gdf, 'url_cocina')
sb.contar_palabras(captions_cocinas, 'kitchen')

266it [03:40,  1.21it/s]


234

In [9]:
captions_banios = sb.generar_descripciones(gdf, 'url_banio')
sb.contar_palabras(captions_banios, 'bathroom')

266it [03:57,  1.12it/s]


250

- De las 266 imágenes identificadas como cocinas por YOLO, BLIP ha coincidido en 234, un 88%. El dato en el caso de los baños mejora, habiendo encontrado 250 coincidencias, un 94%.

- Conocedores de que, habrá falsos negativos y falsos positivos en ambos modelos, el alto porcentaje de coincidencias en ambos casos nos indica que la fiabilidad del primer cribado es alta.

- El siguiente paso será utilizar la API de Anthropic, y específicamente el modelo Sonnet, para asignar una puntuación del 1 al 10 al estado de la estancia, y una estimación de los metros cuadrados, que es una consideración que ha surgido durante el desarrollo y la fase de pruebas del proyecto.

- El prompt de sistema es similar a este:

<div style="background-color:rgb(149, 222, 237); padding: 10px; border-left: 6px solid #000080; color: black; border-radius: 10px;">
You are an AI image analysis system specialized in evaluating property conditions. Your task is to analyze multiple property images in batch and provide a precise evaluation.
Instructions for Analysis:

Analyze each pair of images:

Image 1: Kitchen
Image 2: Bathroom

Evaluation Criteria:

Ratings: Whole numbers 1-5
1: Very poor (complete renovation required)
2: Poor (major renovations needed)
3: Fair (some renovations needed)
4: Good (minor improvements required)
5: Excellent (no renovations needed)

Sizes: Whole numbers in square meters (m²)
</div>

- Como puede verse en el prompt, el resultado de la consulta son 4 valores separados por comas, con la función `analizar_propiedades`, separaremos esos resultados y los asignaremos a 4 nuevas columnas del dataframe. Para tener consultas más eficientes, se enviarán en lotes, o *batches*, que corresponden a grupos de viviendas. Este batch más el dataframe con las URLs identificadas son los parámetros de la función.

In [10]:
gdf.describe()

,codigo,precio,precio_por_zona,tamanio,habitaciones,banios,alquiler_predicho
count,2.660000e+02,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000
mean,1.053132e+08,121159.048872,1760.169173,76.736842,2.293233,1.078947,838.375940
std,5.197263e+06,21279.516803,481.557863,54.852653,0.905180,0.270165,125.886418
min,3.511038e+07,65000.000000,176.000000,34.000000,0.000000,1.000000,617.000000
25%,1.058259e+08,106250.000000,1447.750000,58.000000,2.000000,1.000000,750.250000
50%,1.064768e+08,125000.000000,1776.500000,67.500000,2.000000,1.000000,812.000000
75%,1.068621e+08,139000.000000,2078.500000,81.000000,3.000000,1.000000,899.750000
max,1.070260e+08,150000.000000,2998.000000,850.000000,5.000000,2.000000,1334.000000


In [11]:
df_scoring, resultados = ss.analizar_propiedades(gdf, batch=3)

Procesando lotes:   0%|          | 0/89 [00:00<?, ?it/s]

Error procesando lote: invalid syntax (<string>, line 1)
Error procesando lote: invalid syntax (<string>, line 1)
Error procesando lote: invalid syntax (<string>, line 1)
Error procesando lote: invalid syntax (<string>, line 1)
Error procesando lote: invalid syntax (<string>, line 1)


In [13]:
df_scoring.head(5)

,codigo,precio,precio_por_zona,tipo,exterior,planta,ascensor,tamanio,habitaciones,banios,aire_acondicionado,trastero,terraza,patio,parking,estado,direccion,descripcion,anunciante,contacto,urls_imagenes,distrito,geometry,alquiler_predicho,url_cocina,url_banio,puntuacion_cocina,puntuacion_banio,mts_cocina,mts_banio
0,104792745,149900.0,1180.0,piso,True,4,True,127.0,3,2,False,False,False,True,True,good,"carretera de Huesca, 21","Junto a la Academia General Militar, en carret...","Fincas Ruiz, Jose",876 21 08 84,['https://img4.idealista.com/blur/WEB_LISTING-...,Distrito Rural,POINT (-0.86935 41.6973),1180.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,4,5,12,5
1,106086132,139900.0,1686.0,piso,True,3,True,83.0,2,1,False,False,True,False,False,good,calle de Portugal,Financiación hipoteca 100% ¡oportunidad única ...,"Ciz Inmobiliaria, CIZ",876 21 02 72,['https://img4.idealista.com/blur/WEB_LISTING-...,Delicias,POINT (-0.90232 41.65202),761.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,4,4,10,5
2,105844791,149900.0,1162.0,piso,True,4,True,129.0,3,2,False,False,False,False,True,good,carretera de Huesca,Financiación hipoteca 100%. ¡oportunidad única...,"Ciz Inmobiliaria, CIZ",876 21 02 72,['https://img4.idealista.com/blur/WEB_LISTING-...,Distrito Rural,POINT (-0.87 41.69848),1186.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,4,4,12,5
3,105947628,145000.0,2164.0,piso,True,3,True,67.0,3,1,False,False,False,False,False,good,calle del Monasterio de Solesmes,""" VIVIENDA CON REFORMA A ESTRENAR EN CALLE TRA...","Fincas Ruiz, Pilar Tirapo",876 21 04 93,['https://img4.idealista.com/blur/WEB_LISTING-...,Las Fuentes,POINT (-0.85989 41.64311),798.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,4,5,12,3
4,106889524,125000.0,1984.0,piso,False,7,True,63.0,2,1,False,False,True,False,False,good,calle de Alonso V,"""VIVIENDA EN EL CENTRO EN LA CALLE ALONSO V, C...","Fincas Ruiz, Fincas Ruiz",876 21 04 93,['https://img4.idealista.com/blur/WEB_LISTING-...,Casco Histórico,POINT (-0.87231 41.65146),889.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,2,3,14,5


In [16]:
sm.eliminar_coleccion(bd, 'ventafinal')
sm.subir_geodataframe_a_mongo(bd, df_scoring, 'ventafinal')

GeoDataFrame subido a la colección: ventafinal


In [5]:
df_importado = pd.read_pickle("../data/transformed/final_tags.pkl")
df_importado.shape[0]

278